# Análise de demonstrativos financeiros
## Indicadores de Endividamento

## 1. Carrega os pacotes

In [15]:
# Carrega as bibliotecas
!pip install wget
import wget # biblioteca para realizar o download de sites
import pandas as pd # biblioteca para manipulação de dados e data frames
from zipfile import ZipFile # biblioteca para extrair arquivo zipados

## 2. Cria função para coletar demonstrativos da CVM

In [16]:
def download_extract_concatenate_dfp_files(start_year, end_year, financial_statements):
    """
    Função para realizar o download, extração e concatenação de arquivos zipados contendo os dados da Demonstração Financeira
    Padronizada (DFP) de companhias abertas disponibilizados pela Comissão de Valores Mobiliários (CVM).

    Parâmetros:
    start_year (int): Ano inicial da coleta dos dados.
    end_year (int): Ano final da coleta dos dados.
    financial_statements (list): Lista de nomes dos arquivos de demonstrativos financeiros que deseja-se concatenar. Consolidado: BPA_con (balanço patrimonial ativo);
     BPP_con (Balanço patrimonial passivo) DRE_con; DFC_MI_con; DFC_MD_CON; DMPL_con.


    Retorna:
    Arquivo .csv no diretório principal
    """
    url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"
    # Cria uma lista vazia para inserir os nomes dos arquivos zipados
    arquivo_zipado = []

    # Define os nomes dos arquivos zipados de acordo com o range de datas
    for ano in range(start_year, end_year + 1):
        arquivo_zipado.append(f'dfp_cia_aberta_{ano}.zip')

    # Realiza o download dos arquivos zipados de acordo com a url base
    for arquivos in arquivo_zipado:
        wget.download(url + arquivos)

    # Extrai os arquivos zipados
    for arquivos in arquivo_zipado:
        ZipFile(arquivos, 'r').extractall('DFP')

    # Concatena os dados dos demonstrativos financeiros em um único DataFrame
    for demons in financial_statements:
        arquivo_demonstrativo = pd.DataFrame()
        for ano in range(start_year, end_year + 1):
            arquivo_demonstrativo = pd.concat([arquivo_demonstrativo, pd.read_csv(f'DFP/dfp_cia_aberta_{demons}_{ano}.csv', sep = ';', decimal = ',', encoding = 'ISO-8859-1')])
        arquivo_demonstrativo.to_csv(f'dfp_cia_aberta_{demons}_{start_year}-{end_year}.csv', index = False)

## 3. Coleta os dados

In [17]:
# Coleta os dados
download_extract_concatenate_dfp_files(2010, 2023, ['BPA_con', 'BPP_con', 'DRE_con'])

In [18]:
# Realiza a leitura do arquivo
bpa = pd.read_csv('/content/dfp_cia_aberta_BPA_con_2010-2023.csv')

In [19]:
# Realiza a leitura do arquivo
bpp = pd.read_csv('/content/dfp_cia_aberta_BPP_con_2010-2023.csv')

In [20]:
# Realiza a leitura do arquivo
dre = pd.read_csv('/content/dfp_cia_aberta_DRE_con_2010-2023.csv')

## 4. Tratamento de dados

In [21]:
# Empilha os dfs
dados = pd.concat([bpa, bpp, dre], axis = 0)

# Realiza o filtro das contas contábeis, do código CVM e da ordem do exercício
dados = dados[dados.CD_CONTA.isin(["1", # Ativo Total
                                "2.01", # Passivo Circulante
                                "2.02", # Passivo não circulante
                                "2.01.04", # Empréstimos e Financiamento de CP
                                "2.02.01", # Empréstimos e Financiamento de LP
                                "2.03", # Patrimônio Líquido
                                "3.05" # EBIT
                                 ]) & (dados.CD_CVM == 14311) & (dados.ORDEM_EXERC == 'ÚLTIMO')]
# Seleciona as colunas
dados = dados[["DT_REFER", "CD_CONTA", "DS_CONTA", "VL_CONTA"]]

In [22]:
# Converte a coluna de data para datetime
dados['DT_REFER'] = pd.to_datetime(dados['DT_REFER'])

# Cria a coluna Ano
dados['Ano'] = dados['DT_REFER'].dt.year

dados_pivot = dados.pivot(index = 'Ano', columns = 'CD_CONTA', values = 'VL_CONTA')

In [23]:
dados_pivot

CD_CONTA,1,2.01,2.01.04,2.02,2.02.01,2.03,3.05
Ano,,,,,,,
2010,17859432.0,2536801.0,704252.0,4026805.0,1280982.0,11295826.0,1032307.0
2011,19121663.0,2058821.0,116487.0,4993314.0,2057985.0,12069528.0,1359148.0
2012,21211554.0,2847818.0,274009.0,5866238.0,2987546.0,12497498.0,1000519.0
2013,23111445.0,3347885.0,1014568.0,6834808.0,3517161.0,12928752.0,1226193.0
2014,25618142.0,4055393.0,1299117.0,7879969.0,4755281.0,13682780.0,1709914.0
2015,28947657.0,4789118.0,1232563.0,9574061.0,6528425.0,14584478.0,1908738.0
2016,30434209.0,5656036.0,2601940.0,9622727.0,6235162.0,15155446.0,2044102.0
2017,33162377.0,6109914.0,2416728.0,11541960.0,7413755.0,15510503.0,2141381.0
2018,35930100.0,6695114.0,3297928.0,12898772.0,8267510.0,16336214.0,2394047.0


## 5. Cria os indicadores

In [24]:
# Cria os indicadores
dados_pivot = (
  dados_pivot
 .assign(divida_pl = (dados_pivot["2.01.04"] + dados_pivot["2.02.01"]) / dados_pivot["2.03"],
                divida_ativos = (dados_pivot["2.01.04"] + dados_pivot["2.02.01"]) / dados_pivot["1"],
                divida_ebit = (dados_pivot["2.01.04"] + dados_pivot["2.02.01"]) / dados_pivot["3.05"],
                pl_ativos = dados_pivot["2.03"] / dados_pivot["1"],
                passivos_ativos =   (dados_pivot["2.01"] + dados_pivot["2.02"]) / dados_pivot["1"])
  )

# Seleciona colunas e "reinicia" o índice
dados_endividamento = dados_pivot[['divida_pl', 'divida_ativos', 'divida_ebit', 'pl_ativos', 'passivos_ativos']].reset_index()

In [25]:
dados_pivot

CD_CONTA,1,2.01,2.01.04,2.02,2.02.01,2.03,3.05,divida_pl,divida_ativos,divida_ebit,pl_ativos,passivos_ativos
Ano,,,,,,,,,,,,
2010,17859432.0,2536801.0,704252.0,4026805.0,1280982.0,11295826.0,1032307.0,0.175749,0.111159,1.923104,0.632485,0.367515
2011,19121663.0,2058821.0,116487.0,4993314.0,2057985.0,12069528.0,1359148.0,0.180162,0.113718,1.599879,0.631197,0.368803
2012,21211554.0,2847818.0,274009.0,5866238.0,2987546.0,12497498.0,1000519.0,0.260977,0.153763,3.259863,0.589184,0.410816
2013,23111445.0,3347885.0,1014568.0,6834808.0,3517161.0,12928752.0,1226193.0,0.350516,0.196082,3.695771,0.559409,0.440591
2014,25618142.0,4055393.0,1299117.0,7879969.0,4755281.0,13682780.0,1709914.0,0.442483,0.236332,3.540762,0.534105,0.465895
2015,28947657.0,4789118.0,1232563.0,9574061.0,6528425.0,14584478.0,1908738.0,0.532140,0.268104,4.066031,0.503822,0.496178
2016,30434209.0,5656036.0,2601940.0,9622727.0,6235162.0,15155446.0,2044102.0,0.583097,0.290367,4.323220,0.497974,0.502026
2017,33162377.0,6109914.0,2416728.0,11541960.0,7413755.0,15510503.0,2141381.0,0.633795,0.296435,4.590721,0.467714,0.532286
2018,35930100.0,6695114.0,3297928.0,12898772.0,8267510.0,16336214.0,2394047.0,0.707963,0.321887,4.830915,0.454667,0.545333


In [26]:
# arrendonda os valores
dados_endividamento = dados_endividamento.round(decimals = 3)

In [27]:
dados_endividamento

CD_CONTA,Ano,divida_pl,divida_ativos,divida_ebit,pl_ativos,passivos_ativos
0,2010,0.176,0.111,1.923,0.632,0.368
1,2011,0.180,0.114,1.600,0.631,0.369
2,2012,0.261,0.154,3.260,0.589,0.411
3,2013,0.351,0.196,3.696,0.559,0.441
4,2014,0.442,0.236,3.541,0.534,0.466
5,2015,0.532,0.268,4.066,0.504,0.496
6,2016,0.583,0.290,4.323,0.498,0.502
7,2017,0.634,0.296,4.591,0.468,0.532
8,2018,0.708,0.322,4.831,0.455,0.545
9,2019,0.658,0.302,3.627,0.459,0.541


In [28]:
import plotly.graph_objects as go

# cria uma tabela com os valores
fig = go.Figure(data = [go.Table( # cabeçalho
                                  header = dict(values = list(dados_endividamento.columns),
                                          align = 'left'),
                                  # células
                                  cells = dict(values = [dados_endividamento.Ano, dados_endividamento.divida_pl, dados_endividamento.divida_ativos,
                                                         dados_endividamento.divida_ebit, dados_endividamento.pl_ativos, dados_endividamento.passivos_ativos],
                                          align = 'left')
                                  )
                          ]
                )

fig.show()